In [2]:
#######################################################################
#
# Set up the variables that store all the data.
#
# files contains an array of tuples of (tif_file, zip_file). It assumes
# that all the files are stored in a directory called data, with each
# file named 'AMG[1-3]_exp[1-4].[tif/zip]', as mentioned in the dataset
# given to us by Jan
#
# data is an array of (stacks, rois) corresponding to (tif, zip) files.
# Each stack and roi is loaded using load_stacks and load_roi, and is
# a 3D array of size (num_frames/num_rois)*512*512
#
#######################################################################

%matplotlib inline

from PIL import Image, ImageDraw
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
from scipy.ndimage.filters import laplace
from skimage import feature


# https://gist.github.com/luispedro/3437255
# Copyright: Luis Pedro Coelho <luis@luispedro.org>, 2012
# License: MIT

def read_roi(fileobj):
# This is based on:
# http://rsbweb.nih.gov/ij/developer/source/ij/io/RoiDecoder.java.html
# http://rsbweb.nih.gov/ij/developer/source/ij/io/RoiEncoder.java.html


    SPLINE_FIT = 1
    DOUBLE_HEADED = 2
    OUTLINE = 4
    OVERLAY_LABELS = 8
    OVERLAY_NAMES = 16
    OVERLAY_BACKGROUNDS = 32
    OVERLAY_BOLD = 64
    SUB_PIXEL_RESOLUTION = 128
    DRAW_OFFSET = 256


    pos = [4]
    def get8():
        pos[0] += 1
        s = fileobj.read(1)
        if not s:
            raise IOError('readroi: Unexpected EOF')
        return ord(s)

    def get16():
        b0 = get8()
        b1 = get8()
        return (b0 << 8) | b1

    def get32():
        s0 = get16()
        s1 = get16()
        return (s0 << 16) | s1

    def getfloat():
        v = np.int32(get32())
        return v.view(np.float32)

    magic = fileobj.read(4)
    if magic != b'Iout':
        raise IOError('Magic number not found. Value = ' + str(magic))
    version = get16()

    # It seems that the roi type field occupies 2 Bytes, but only one is used
    roi_type = get8()
    # Discard second Byte:
    get8()

    if not (0 <= roi_type < 11):
        raise ValueError('roireader: ROI type %s not supported' % roi_type)

    if roi_type != 7:
        raise ValueError('roireader: ROI type %s not supported (!= 7)' % roi_type)

    top = get16()
    left = get16()
    bottom = get16()
    right = get16()
    n_coordinates = get16()

    x1 = getfloat() 
    y1 = getfloat() 
    x2 = getfloat() 
    y2 = getfloat()
    stroke_width = get16()
    shape_roi_size = get32()
    stroke_color = get32()
    fill_color = get32()
    subtype = get16()
    if subtype != 0:
        raise ValueError('roireader: ROI subtype %s not supported (!= 0)' % subtype)
    options = get16()
    arrow_style = get8()
    arrow_head_size = get8()
    rect_arc_size = get16()
    position = get32()
    header2offset = get32()

    if options & SUB_PIXEL_RESOLUTION:
        getc = getfloat
        points = np.empty((n_coordinates, 2), dtype=np.float32)
    else:
        getc = get16
        points = np.empty((n_coordinates, 2), dtype=np.int16)
    points[:,1] = [getc() for i in range(n_coordinates)]
    points[:,0] = [getc() for i in range(n_coordinates)]
    points[:,1] += left
    points[:,0] += top
    points -= 1
    return points

def read_roi_zip(fname):
    import zipfile
    zf = zipfile.ZipFile(fname)
    x = []
    for n in zf.namelist():
        x.append(read_roi(zf.open(n)))
    zf.close()
    return x
    
# load_stacks takes in the tif file, and returns a 3D numpy array of 
# time*xdimension*ydimension
# tif -> (time,w,h)
def load_stacks(path):
    im = Image.open(path)
    stk = []
    while True:
        stk.append(np.array(im))
        try:
            im.seek(len(stk))
        except EOFError:
            break
    return np.array(stk, dtype='float32')

# load_rois takes in a zip file containing rois, width and height
# (usually 512*512 for the images given), as well as optional args
# for fill, x displacement, and y displacement (by default 1, 0, 0).
# It returns a numpy array of 0-1 matrices, where each matrix holds 1
# roi. An roi is visible as a roughly sphere-shaped area, usually 
# between 100 - 350 pixels, of 1s. The rest of each 512*512 matrix
# is 0s
# roi zip -> (index,w,h)
def load_roi(path, width, height, fill=1, xdisp=0, ydisp=0):
    rois = read_roi_zip(path)
    ret = []
    for i,roi in enumerate(rois):
        poly = []
        for x in roi:
            poly.append((x[1],x[0]))
        img = Image.new('L', (width, height), 0)
        ImageDraw.Draw(img).polygon(poly, outline=1, fill=fill)
        ret.append(np.array(img))
    return np.array(ret,dtype='float32')


plt.rcParams['figure.figsize'] = (16, 16)

# generate filenames
files = []
for i in range(1,4):
    for j in range(1,5):
        prefix = 'data/AMG%d_exp%d'%(i,j)
        files.append( (prefix+'.tif', prefix+'.zip') )
        
# load data
data = []
for i,(s,r) in enumerate(files):
    if i==8: # Stack 8 has offset rois
        data.append((load_stacks(s), load_roi(r, 512, 512, xdisp=9, ydisp=-1)))
    else:
        data.append((load_stacks(s), load_roi(r, 512, 512)))


In [3]:
#######################################################################
#
# Create the scoring code. 
# Running score(None, None, true_rois, pred_rois) will return a score
# object (where true_rois and pred_rois are lists of 2D 0-1 arrays)
#
#######################################################################

import matplotlib.pyplot as plt
import numpy as np
from itertools import chain
import random

class TestSet: 
    """Stores test set or validation set data and images.
    
    self.data   = list of 3D image arrays with dimensions (stack, time, w, h)
    self.labels = list of 3D ROI arrays with dimensions (stack, ROI index, w, h)
    Change code so "files" points to the test/validation set
    """
    def __init__(self):
        files = []
        for j in range(3,5):
            prefix = 'data/AMG3_exp%d'%(j)
            files.append( (prefix+'.tif', prefix+'.zip') )
        self.data = []
        self.labels = []
        for i,(s,r) in enumerate(files):
            self.data.append(load_stack(s))
            self.labels.append(load_rois(r, 512, 512))

class Score:
    """Class to calculate and store score.
    
    Constructor takes a prediction function that takes a list of image stacks and 
    returns a list of 3D 0-1 ROI arrays with dimensions (stack, roi index, w, h).
    
    self.predictions      = list of predicted ROIs (stack, ROI index, w, h)
    self.categorized      = list of predicted ROIs divided into FPs, FNs, and TPs matched with true ROIs
                             pred ROIs with <0.5 overlap to any true ROI are false positives
                             true ROIs with <0.5 overlap to any pred ROI are false negatives
                             true positive pairs are made by maximizing overlap between pairs
                             e.g. self.categorized[0]["fps"][n] = nth false positive ROI from the 0th image stack
                             e.g. self.categorized[0]["tps"][n] = nth (pred ROI, true ROI) pair from 0th image stack
    self.total_f1_score   = F1 score with all image stacks merged
    self.f1_scores        = list of F1 scores per stack
    self.total_precision  = total precision with all image stacks merged
    self.precisions       = list of precisions per stack
    self.total_recall     = total recall with all stacks merged
    self.recalls          = list of recalls per stack
    self.total_overlap_bq = dict of pixel-wise precisions and recalls for all paired TP predicted ROIs and true ROIs
                            keys = ["mean precision", "std precision", "mean recall", "std recall"]
                             e.g. self.total_overlap_bq["mean precision"] = 
                               mean (pixels in both pred_roi and true_roi) / (pixels in pred_roi)
    self.overlap_bqs      = list of dicts as in self.total_overlap_bq with one dict per image stack                   
    """
    def __init__(self, prediction_func, data, actual_labels, predicted_labels=None):
        if predicted_labels is None:
            self.predictions = prediction_func(data)
        else:
            self.predictions = predicted_labels
        # assert that predicted and true ROI stacks are the same shape
        assert(self.predictions[0].shape[1] == actual_labels[0].shape[1])
        # assert that predicted ROI stacks are 0-1 arrays
        for i in range(len(self.predictions)):
            assert(np.all(np.logical_or(self.predictions[i] == 1, self.predictions[i] == 0)))
        self.categorized = categorize(self.predictions, actual_labels)
        self.precisions, self.total_precision, self.recalls, self.total_recall = calc_precision_recall(self.categorized)
        self.f1_scores = list(map(calc_f1_score, zip(self.precisions, self.recalls)))
        self.total_f1_score = calc_f1_score((self.total_precision, self.total_recall))
        self.overlap_bqs, self.total_overlap_bq = overlap_boundary_quality(self.categorized)
       
    def __str__(self):
        """Convert Score to readable output"""
        string =  "Total F1 Score      = {}\n".format(self.total_f1_score)
        string += "F1 Score per stack  = {!s}\n".format(self.f1_scores)
        string += "Total Precision     = {}\n".format(self.total_precision)
        string += "Precision per stack = {!s}\n".format(self.precisions)
        string += "Total Recall        = {}\n".format(self.total_recall)
        string += "Recall per stack    = {!s}\n".format(self.recalls)
        string += "Overlap Boundary Quality, all stacks = {!s}\n".format(self.total_overlap_bq)
        string += "Overlap Boundary Quality, per stack  = {!s}\n\n".format(self.overlap_bqs)
        return string
    
    def plot(self):
        """Plot all data in Score"""
        fig, ax = plt.subplots()
        yvals = [self.total_f1_score, self.total_precision, self.total_recall, 
                self.total_overlap_bq["mean precision"], self.total_overlap_bq["std precision"],
                self.total_overlap_bq["mean recall"], self.total_overlap_bq["std recall"]]
        xvals = np.arange(len(yvals))
        plt.bar(xvals, yvals, width=0.6)
        ax.set_xticks(xvals + 0.3)
        ax.set_xticklabels(['F1 score', 'precision', 'recall', 'mbp', 'sbp', 'mbr', 'sbr'])
        plt.show()
        
def categorize(predictions, labels):
    "Divide predictions and labels into FPs, FNs, and TPs"
    categorized = []
    rois_pred, rois_true = list(predictions.copy()), list(labels.copy())
    categorized.append({"fps":[], "fns":[], "tps":[]})
    for roi_pred in rois_pred:
        overlaps = list(map(lambda roi_true: calc_overlap(roi_pred, roi_true)[0], rois_true))
        best_overlap, best_index = np.max(overlaps), np.argmax(overlaps)
        if best_overlap > 0.5:
            categorized[0]["tps"].append((roi_pred, rois_true[best_index]))
            del rois_true[best_index]
        else:
            categorized[0]["fps"].append(roi_pred)
    for roi_true in rois_true:
        categorized[0]["fns"].append(roi_true)
    return categorized

def calc_precision_recall(categorized):
    """Calculate precision and recall from categorized predictions and labels"""
    num_fps = [len(categorized[i]["fps"]) for i in range(len(categorized))]
    num_fns = [len(categorized[i]["fns"]) for i in range(len(categorized))]
    num_pairs = [len(categorized[i]["tps"]) for i in range(len(categorized))]
    precisions = [num_pairs[i] / float(num_pairs[i] + num_fps[i]) for i in range(len(categorized))]
    recalls = [num_pairs[i] / float(num_pairs[i] + num_fns[i]) for i in range(len(categorized))]
    total_precision = sum(num_pairs) / float(sum(num_pairs) + sum(num_fps))
    total_recall = sum(num_pairs) / float(sum(num_pairs) + sum(num_fns))
    return precisions, total_precision, recalls, total_recall
    
def calc_overlap(roi_pred, roi_true):
    """For a given (2D array, 2D array) pair of predicted and true ROIs,
    overlap   = (# pixel intersection / (# pixel union)
    precision = (# pixel intersection) / (# pixels in true ROI)
    recall    = (# pixel interection) / (# pixels in pred ROI)
    """

    intersection = float(np.dot(roi_pred.flat, roi_true.flat))
    
    if intersection == 0: 
        return 0, 0, 0
    
    pred = np.sum(roi_pred)
    true = np.sum(roi_true)
    union = pred + true - intersection
    
    precision = intersection / pred
    recall = intersection / true
    overlap = intersection / union
    return overlap, precision, recall

def calc_f1_score(pr):
    """F1 score is harmonic mean of precision and recall"""
    precision, recall = pr
    if recall == 0:
        return 0 
    else:
        return (2 * precision * recall) / (precision + recall)
    
def overlap_boundary_quality(categorized):
    """Calculates pixel-based mean and std of precision and recall 
    of TP (pred roi, true roi) pairs
    """
    qualities = []
    precisions = []
    recalls = []
    for i in range(len(categorized)):
        precisions.append([])
        recalls.append([])
        for roi_pred, roi_true in categorized[i]["tps"]:
            _, precision, recall = calc_overlap(roi_pred, roi_true)
            precisions[i].append(precision)
            recalls[i].append(recall)
        qualities.append({"mean precision": np.mean(precisions[i]), "std precision": np.std(precisions[i]),
                          "mean recall": np.mean(recalls[i]), "std recall": np.std(recalls[i])})
    overall = {"mean precision": np.mean(list(chain.from_iterable(precisions))),
               "std precision": np.std(list(chain.from_iterable(precisions))),
               "mean recall": np.mean(list(chain.from_iterable(recalls))),
               "std recall": np.std(list(chain.from_iterable(recalls)))}
    return qualities, overall   

def plot_multiple_scores(scores):
    """Takes list of multiple scores and plots precision/recall curves
    and f1 score comparisons.
    """
    f1_fig, ax = plt.subplots()
    f1s = [s.total_f1_score for s in scores]
    x = np.arange(len(f1s))
    plt.bar(x, f1s, width=1)
    ax.set_xticks(x + 0.5)
    ax.set_xticklabels(map(str,x))
    plt.ylabel('F1 score')
    plt.xlabel('Algorithm Number')
    
    pr_fig = plt.figure()
    recalls = [s.total_recall for s in scores]
    precisions = [s.total_precision for s in scores]
    r, p, num = zip(*sorted(zip(recalls, precisions, np.arange(len(recalls)))))
    plt.plot(r, p, '.-', markersize=10)
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    print("Score Number in order of increasing recall: " + str(num))
    
    pr_boundary_fig = plt.figure()
    recalls = [s.total_overlap_bq["mean recall"] for s in scores]
    precisions = [s.total_overlap_bq["mean precision"] for s in scores]
    r_std = [s.total_overlap_bq["std recall"] for s in scores]
    p_std = [s.total_overlap_bq["std precision"] for s in scores]
    rm, pm, rs, ps, num = zip(*sorted(zip(recalls, precisions, r_std, p_std, np.arange(len(recalls)))))
    plt.errorbar(rm, pm, xerr=rs, yerr=ps, fmt='.-',markersize=10)
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    print("Score number in order of increasing boundary recall: " + str(num))
    
    plt.show()


In [9]:

import pickle
from scipy.ndimage import zoom


# Takes in the stack to be tested. Returns a tuple of the original image, the true rois,
# the raw SVM predictions and the Convolutional net predictions
def getStackValues(stack_num):
    (stk, rois) = data[stack_num]
    M = stk.max(axis=0) - stk.std(axis=0)
    M = np.clip((M-900)/400, 0, 1 )
    M**=0.5
    R = rois.max(axis=0) 

    SVMTest = np.load("svmmap" + str(stack_num) + ".npy")
    cnn_test_outputs = pickle.load(open("cnn_results.pickle", 'rb'), encoding="latin1")
    CNNTest = zoom(cnn_test_outputs[stack_num], 2, order=0)
    CNNTest = np.clip((CNNTest+1)/2, 0, 1)
    
    return (M, R, SVMTest, CNNTest)


# Takes in the two predictions, the original image,  and the desired confidence thresholding
# level. Combines them into one output stack of 0-1 pixels
def combineOutputs(SVMTest, CNNTest, M, min_confidence):
    out_pixels = (SVMTest + CNNTest)/2
    
    # False negatives occur at blood vessels a lot, however blood vessels usually correspond to 
    # lower pixel values than neurons. We can get rid of a lot of false positives, while losing
    # very few true positives, by setting a thershold for the minimum pixel brightness of a 
    # true positive
    true_positive_brightness = 0.15
    out_pixels[np.where(M < true_positive_brightness)] = 0
    
    # Convert out_pixels to a 0-1 array
    out_pixels[np.where(out_pixels > min_confidence)] = 1
    out_pixels[np.where(out_pixels <= min_confidence)] = 0
    return out_pixels
    

# Takes in the pixel predictions, the actual image and the real rois. Prints pixel stats and
# visualizes the image
def viewPixelStats(out_pixels, M, R): 
    K = np.zeros((512,512,3))
    K[:,:,0] = K[:,:,1] = K[:,:,2] = M
    false_pos = []
    true_pos = []
    false_neg = []

    for i in range(dimensions):
        for j in range(dimensions):
            if R[i,j] == 1 and out_pixels[i,j] > 0.8:
                K[i,j,0] = 0
                K[i,j,1] = 1
                K[i,j,2] = 0
                true_pos.append(out_pixels[i,j])
            elif R[i,j] == 1:
                K[i,j,0] = 1
                K[i,j,1] = 0
                K[i,j,2] = 0
                false_neg.append(out_pixels[i,j])
            elif out_pixels[i,j] > 0.8:
                K[i,j,0] = 0
                K[i,j,1] = 0
                K[i,j,2] = 1
                false_pos.append(out_pixels[i,j])

    print("True pos pixels: " + str(np.size(true_pos)))
    print("False pos pixels: " + str(np.size(false_pos)))
    print("False neg pixels: " + str(np.size(false_neg)))
    print("Precision: " + str(np.size(true_pos)/(np.size(true_pos) + np.size(false_pos))))
    print("Recall: " + str(np.size(true_pos)/(np.size(true_pos) + np.size(false_neg))))
    plt.imshow(K)
    
# Takes in the pixel predictions as a 0-1 2D array, and returns a list of 2D 0-1 arrays,
# where each array consists of 1 ROI (indicated by 1s) surrounded by 0s
def get3DRois(out_pixels):
    # Array of rois to return
    out_rois = []
    
    # The dimensions of the square image
    dimensions = 512

    # An array to maintain the locations of all the rois in a 2D 0-1 matrix
    flat_rois = np.zeros((dimensions,dimensions))

    # The region at the center of a neuron to consider when combining neurons
    rad = 2

    # Threshold can be used to adjust the precision vs accuracy. Higher threshold
    # leads to high precision but lower accuracy. 
    threshold = 0.05

    # Function that takes in the map of all current rois, and a potential new
    # roi, and returns true if there is no overlap between the existing and new
    # rois
    def AvoidOverlaps(current_rois, potential_roi):
        return np.sum(potential_roi[np.where(current_rois == 1)]) == 0

    # Returns probability that a box of size 2*r in out_pixels centered on (i, j) contains a neuron
    def NeuronProb(i, j, r):
        return np.sum(out_pixels[i-r:i+r, j-r:j+r])/((2*r)**2)

    # Adds an roi centered around i,j to out_rois, and removes it from out_pixels
    def AddRoi(i, j):
        # Minimum, maximum radius of the potential roi
        r = 8
        # Add new roi
        patch = np.zeros((dimensions, dimensions))
        # Make it a sphere
        for x in range(0, 2*r):
            for y in range(0, 2*r):
                if (r-x)**2 + (r-y)**2 <= r*r:
                    patch[i-r+x, j-r+y] = 1
        if AvoidOverlaps(flat_rois, patch):
            out_rois.append(patch)
            # Remove 1s from that region
            out_pixels[np.where(patch == 1)] = 0
            # Add 1s to the output region
            flat_rois[np.where(patch == 1)] = 1

    # Extract rois
    for i in range(rad, dimensions-rad-1):
        for j in range(rad, dimensions-rad-1):
            prob = NeuronProb(i, j, rad)
            # Check right and bottom for continuation of same neuron
            if prob > threshold and prob > NeuronProb(i, j+1, rad) and prob > NeuronProb(i+1, j, rad):
                AddRoi(i,j)
    return out_rois

# View the predicted rois overlaid over the original image and the predicted rois 
def viewPredictedRois(out_rois, M, R):
    K = np.zeros((512,512,3))
    K[:,:,0] = K[:,:,1] = K[:,:,2] = M
    K[:,:,0] = np.maximum(M, R)
    K[:,:,2] = np.maximum(M, np.array(out_rois).max(axis=0))
    plt.imshow(K)


# Runs the scoring code and prints and visualizes the score, given the true
# and predicted roi stacks
def viewScore(true_rois, pred_rois):
    score = Score(None, None, rois, np.array(out_rois))
    print(score)
    score.plot()
    


In [ ]:

# CHOOSE THE SET TO BE TESTED
test_set = 11

# CHOOSE THE MIN CONFIDENCE THRESHOLD
min_confidence = 0.9

(stk, rois) = data[test_set]
(M, R, SVMTest, CNNTest) = getStackValues(test_set)
out_pixels = combineOutputs(SVMTest, CNNTest, M, min_confidence)
# viewPixelStats(out_pixels, M, R)
out_rois = get3DRois(out_pixels)
# viewPredictedRois(out_rois, M, R)
viewScore(rois, np.array(out_rois))
